Random forest preparation:
Below is the code I used for the IMD data for the FBA c/w - will be different as I am doing a classifiction rather than a regression and will use scikit rather than sklearn

Also, for the FBA, there were 7 input features (OSM info) and 1 output (IMD score). For this there will be up to 358 input features (spectral bands) and 1 output (PFT). Therefore will need a table with all of the info in - make by creating a vector file of the bsq? - test this.

Want to iterate through the data rather than adding manually (358 bands)

Then need to split into X (all the bands) and y (PFT)

When complete, try a range of proportions of training and test data and remember to cross validate results.

Also possibly use the k-means clustering to validate and compare results

In [2]:
cd /mnt/c/Users/rache/OneDrive - The University of Nottingham/MRes/Flow_Country_HSI_and_PSM/python_notebooks

/mnt/c/Users/rache/OneDrive - The University of Nottingham/MRes/Flow_Country_HSI_and_PSM/python_notebooks


In [3]:
%load_ext autoreload
%autoreload 2

In [22]:
import importlib
import hyperspectral_machine_learning as hml

from hyperspectral_machine_learning.random_forest import original_data_transformation, derivative_data_transformation

importlib.reload(hml)

# hml.random_forest.original_to_geo_dataframes('restored_2006_6_classes_for_ML.csv', 0.3, max_depth = 5)
# hml.train_test_data.x_and_y_train_test_vis('transformed_spectral_signatures/first_derivative_98_sample.csv', test_size=0.3)


# hml.random_forest.random_forest_results(hml.random_forest.original_data_transformation('original', 'restored_2006_6_classes_for_ML.csv'), test_size=0.3, max_depth=5)
# will need to update with the transformations for the train_test dataset
data_transformations = [
        original_data_transformation('original', 'train_test_data_full.csv', 'erosion_gdf2.csv'),
        derivative_data_transformation('first derivative', 'transformed_spectral_signatures/first_derivative_98_sample_train_test.csv', 'transformed_spectral_signatures/first_derivative_erosion.csv'),
        derivative_data_transformation('second derivative', 'transformed_spectral_signatures/second_derivative_98_sample_train_test.csv', 'transformed_spectral_signatures/second_derivative_erosion.csv'),
        derivative_data_transformation('continuum removal', 'transformed_spectral_signatures/BDNA_train_test.csv', 'transformed_spectral_signatures/BDNA_train_test_erosion.csv')
    ]


hml.random_forest.random_forest_csv_and_shape_file_outputs('erosion_updated', data_transformations, test_size=0.3, max_depth=5, output_directory='outputs')
# hml.random_forest.random_forest_results(data_transformations, complete_dataset, test_size=0.3, max_depth=5)

# data_transformations # when run, pft returns a number. Also, how does the data_transformations code run when there is no hml.random...

/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 0
- 1
- 10
- 100
- 101
- ...
Feature names seen at fit time, yet now missing:
- band1
- band10
- band100
- band101
- band102
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 10
- 11
- 12
- 13
- 14
- ...
Feature names seen at fit time, yet now missing:
- band10
- band11
- band12
- band13
- band14
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feat

,model_result,spectrum,data_transformation,accuracy_mean,accuracy_standard_deviation
0,POINTID geometry...,full,original,0.679185,0.014260
1,POINTID geometry...,VIS,original,0.607071,0.034715
2,POINTID geometry...,RE,original,0.563618,0.019642
3,POINTID geometry...,NIR,original,0.572239,0.005186
4,POINTID geometry...,SWIR,original,0.589630,0.009227
5,geometry clf_pr...,full,first derivative,0.710870,0.042266
6,geometry clf_pr...,VIS,first derivative,0.598176,0.023132
7,geometry clf_pr...,RE,first derivative,0.514393,0.031755
8,geometry clf_pr...,NIR,first derivative,0.586732,0.015603
9,geometry clf_pr...,SWIR,first derivative,0.624188,0.030557


In [23]:
hml.random_forest.random_forest_csv_and_shape_file_outputs('erosion_updated', data_transformations, test_size=0.25, max_depth=5, output_directory='outputs')

/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 0
- 1
- 10
- 100
- 101
- ...
Feature names seen at fit time, yet now missing:
- band1
- band10
- band100
- band101
- band102
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 10
- 11
- 12
- 13
- 14
- ...
Feature names seen at fit time, yet now missing:
- band10
- band11
- band12
- band13
- band14
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feat

,model_result,spectrum,data_transformation,accuracy_mean,accuracy_standard_deviation
0,POINTID geometry...,full,original,0.630978,0.011593
1,POINTID geometry...,VIS,original,0.593213,0.028082
2,POINTID geometry...,RE,original,0.530928,0.021869
3,POINTID geometry...,NIR,original,0.530999,0.018305
4,POINTID geometry...,SWIR,original,0.558849,0.057634
5,geometry clf_pr...,full,first derivative,0.641323,0.043641
6,geometry clf_pr...,VIS,first derivative,0.500143,0.047926
7,geometry clf_pr...,RE,first derivative,0.482567,0.039458
8,geometry clf_pr...,NIR,first derivative,0.572380,0.018227
9,geometry clf_pr...,SWIR,first derivative,0.606995,0.029099


In [24]:
hml.random_forest.random_forest_csv_and_shape_file_outputs('erosion_updated', data_transformations, test_size=0.3, max_depth=4, output_directory='outputs')

/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 0
- 1
- 10
- 100
- 101
- ...
Feature names seen at fit time, yet now missing:
- band1
- band10
- band100
- band101
- band102
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 10
- 11
- 12
- 13
- 14
- ...
Feature names seen at fit time, yet now missing:
- band10
- band11
- band12
- band13
- band14
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feat

,model_result,spectrum,data_transformation,accuracy_mean,accuracy_standard_deviation
0,POINTID geometry...,full,original,0.653248,0.017295
1,POINTID geometry...,VIS,original,0.592429,0.044806
2,POINTID geometry...,RE,original,0.569290,0.022972
3,POINTID geometry...,NIR,original,0.586607,0.024360
4,POINTID geometry...,SWIR,original,0.595327,0.015924
5,geometry clf_pr...,full,first derivative,0.699550,0.032173
6,geometry clf_pr...,VIS,first derivative,0.552049,0.009509
7,geometry clf_pr...,RE,first derivative,0.514468,0.011424
8,geometry clf_pr...,NIR,first derivative,0.595527,0.041100
9,geometry clf_pr...,SWIR,first derivative,0.604023,0.015297


In [25]:
hml.random_forest.random_forest_csv_and_shape_file_outputs('erosion_updated', data_transformations, test_size=0.25, max_depth=4, output_directory='outputs')

/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 0
- 1
- 10
- 100
- 101
- ...
Feature names seen at fit time, yet now missing:
- band1
- band10
- band100
- band101
- band102
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 10
- 11
- 12
- 13
- 14
- ...
Feature names seen at fit time, yet now missing:
- band10
- band11
- band12
- band13
- band14
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feat

,model_result,spectrum,data_transformation,accuracy_mean,accuracy_standard_deviation
0,POINTID geometry...,full,original,0.596435,0.044839
1,POINTID geometry...,VIS,original,0.568943,0.006203
2,POINTID geometry...,RE,original,0.544817,0.058965
3,POINTID geometry...,NIR,original,0.499785,0.063899
4,POINTID geometry...,SWIR,original,0.520511,0.037556
5,geometry clf_pr...,full,first derivative,0.617053,0.054063
6,geometry clf_pr...,VIS,first derivative,0.517039,0.048589
7,geometry clf_pr...,RE,first derivative,0.458512,0.023764
8,geometry clf_pr...,NIR,first derivative,0.606744,0.035527
9,geometry clf_pr...,SWIR,first derivative,0.555126,0.010406


In [26]:
hml.random_forest.random_forest_csv_and_shape_file_outputs('erosion_updated', data_transformations, test_size=0.3, max_depth=5, output_directory='outputs', cv=5)

/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 0
- 1
- 10
- 100
- 101
- ...
Feature names seen at fit time, yet now missing:
- band1
- band10
- band100
- band101
- band102
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 10
- 11
- 12
- 13
- 14
- ...
Feature names seen at fit time, yet now missing:
- band10
- band11
- band12
- band13
- band14
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feat

,model_result,spectrum,data_transformation,accuracy_mean,accuracy_standard_deviation
0,POINTID geometry...,full,original,0.702236,0.060150
1,POINTID geometry...,VIS,original,0.632795,0.052421
2,POINTID geometry...,RE,original,0.609689,0.057297
3,POINTID geometry...,NIR,original,0.583810,0.059675
4,POINTID geometry...,SWIR,original,0.647329,0.020706
5,geometry clf_pr...,full,first derivative,0.731263,0.074596
6,geometry clf_pr...,VIS,first derivative,0.566501,0.042635
7,geometry clf_pr...,RE,first derivative,0.531677,0.049954
8,geometry clf_pr...,NIR,first derivative,0.667453,0.056589
9,geometry clf_pr...,SWIR,first derivative,0.656025,0.046473


In [27]:
hml.random_forest.random_forest_csv_and_shape_file_outputs('erosion_updated', data_transformations, test_size=0.25, max_depth=5, output_directory='outputs', cv=5)

/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 0
- 1
- 10
- 100
- 101
- ...
Feature names seen at fit time, yet now missing:
- band1
- band10
- band100
- band101
- band102
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 10
- 11
- 12
- 13
- 14
- ...
Feature names seen at fit time, yet now missing:
- band10
- band11
- band12
- band13
- band14
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feat

,model_result,spectrum,data_transformation,accuracy_mean,accuracy_standard_deviation
0,POINTID geometry...,full,original,0.744828,0.035166
1,POINTID geometry...,VIS,original,0.648276,0.062260
2,POINTID geometry...,RE,original,0.637931,0.028850
3,POINTID geometry...,NIR,original,0.606897,0.025340
4,POINTID geometry...,SWIR,original,0.606897,0.042792
5,geometry clf_pr...,full,first derivative,0.710345,0.039917
6,geometry clf_pr...,VIS,first derivative,0.582759,0.036818
7,geometry clf_pr...,RE,first derivative,0.527586,0.044428
8,geometry clf_pr...,NIR,first derivative,0.644828,0.077567
9,geometry clf_pr...,SWIR,first derivative,0.634483,0.035166


In [28]:
hml.random_forest.random_forest_csv_and_shape_file_outputs('erosion_updated', data_transformations, test_size=0.3, max_depth=4, output_directory='outputs', cv=5)

/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 0
- 1
- 10
- 100
- 101
- ...
Feature names seen at fit time, yet now missing:
- band1
- band10
- band100
- band101
- band102
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 10
- 11
- 12
- 13
- 14
- ...
Feature names seen at fit time, yet now missing:
- band10
- band11
- band12
- band13
- band14
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feat

,model_result,spectrum,data_transformation,accuracy_mean,accuracy_standard_deviation
0,POINTID geometry...,full,original,0.713789,0.048406
1,POINTID geometry...,VIS,original,0.627288,0.039042
2,POINTID geometry...,RE,original,0.586625,0.032116
3,POINTID geometry...,NIR,original,0.577930,0.046112
4,POINTID geometry...,SWIR,original,0.606832,0.043223
5,geometry clf_pr...,full,first derivative,0.690807,0.031907
6,geometry clf_pr...,VIS,first derivative,0.601242,0.069592
7,geometry clf_pr...,RE,first derivative,0.523230,0.048122
8,geometry clf_pr...,NIR,first derivative,0.641739,0.026222
9,geometry clf_pr...,SWIR,first derivative,0.606832,0.043223


In [29]:
hml.random_forest.random_forest_csv_and_shape_file_outputs('erosion_updated', data_transformations, test_size=0.25, max_depth=4, output_directory='outputs', cv=5)

/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 0
- 1
- 10
- 100
- 101
- ...
Feature names seen at fit time, yet now missing:
- band1
- band10
- band100
- band101
- band102
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 10
- 11
- 12
- 13
- 14
- ...
Feature names seen at fit time, yet now missing:
- band10
- band11
- band12
- band13
- band14
- ...

  warnings.warn(message, FutureWarning)
/home/rzw/.local/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feat

,model_result,spectrum,data_transformation,accuracy_mean,accuracy_standard_deviation
0,POINTID geometry...,full,original,0.682759,0.043069
1,POINTID geometry...,VIS,original,0.593103,0.035502
2,POINTID geometry...,RE,original,0.582759,0.039917
3,POINTID geometry...,NIR,original,0.603448,0.036166
4,POINTID geometry...,SWIR,original,0.555172,0.025340
5,geometry clf_pr...,full,first derivative,0.637931,0.036166
6,geometry clf_pr...,VIS,first derivative,0.520690,0.035166
7,geometry clf_pr...,RE,first derivative,0.517241,0.037774
8,geometry clf_pr...,NIR,first derivative,0.617241,0.029663
9,geometry clf_pr...,SWIR,first derivative,0.572414,0.050444


In [ ]:
hml.Vector.output_csv_for_analysis('../preprocessed_data/dimensionality_reduction/geotiff/cross_lochs.tif', 'cross_lochs_gdf')

# hml.Vector.output_csv_for_analysis('../InSAR/cross_lochs_5_m2.tif/cross_lochs_5_m2.tif.0.tif', '../InSAR/cross_lochs_geometry.csv', 'cross_lochs_insar_gdf')
# won't work as set up for hyperspectral data not InSAR

# hml.Vector.point_id('../preprocessed_data/dimensionality_reduction/geotiff/cross_lochs.tif')

In [ ]:
data_tiff = "../preprocessed_data/dimensionality_reduction/geotiff/cross_lochs.tif"
geom_file = "../preprocessed_data/dimensionality_reduction/Shape/cross_lochs_geometry.csv"
name = "cross_lochs_gdf"

hml.Vector.output_csv_for_analysis(data_tiff, geom_file, name)



In [ ]:
data_vis, data_RE, data_NIR, data_SWIR = hml.spectrum.bands('transformed_spectral_signatures/first_derivative_98_sample.csv')
data_SWIR

In [ ]:
hml.random_forest.random_forest_csv_and_shape_file_outputs('Cross_Lochs', 'cross_lochs_6_classes_for_ML.csv', 'transformed_spectral_signatures/first_derivative_98_sample.csv', 'transformed_spectral_signatures/second_derivative_98_sample.csv', 'transformed_spectral_signatures/BDNA_98_sample.csv', test_size=0.3, max_depth=5)


In [ ]:
import pandas as pd
import geopandas

csvs_to_check = ['../transformed_spectral_signatures/first_derivative_98_sample.csv', '../transformed_spectral_signatures/first_derivative_98_sample.csv', '../transformed_spectral_signatures/first_derivative_98_sample.csv']

csvs = []

for csv_path in csvs_to_check:
    csvs.append(pd.read_csv(csv_path))

# cross_lochs = pd.read_csv('../cross_lochs_6_classes_for_ML.csv')
cross_lochs = pd.read_csv('../transformed_spectral_signatures/first_derivative_98_sample.csv')
# cross_lochs = pd.read_csv('../transformed_spectral_signatures/BDNA_98_sample.csv')

cross_lochs['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs['geometry'])

cross_lochs_gdf = geopandas.GeoDataFrame(cross_lochs, geometry='geometry')

cross_lochs_gdf.head()


In [ ]:
from sklearn.model_selection import train_test_split


train_shrub_sphagnum, test_shrub_sphagnum = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('shrub_sphagnum')], test_size=0.3)
train_water, test_water = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('water')], test_size=0.3)
train_sphagnum_r, test_sphagnum_r = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('spahgnum_r')], test_size=0.3)
train_pool_bogbean, test_pool_bogbean = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('pool_bogbean')], test_size=0.3)
train_calluna_mix, test_calluna_mix = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('calluna_mix')], test_size=0.3)
train_rushes_sedges, test_rushes_sedges = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('rushes_sedges')], test_size=0.3)


train_frames = [train_shrub_sphagnum, train_water, train_sphagnum_r, train_pool_bogbean, train_calluna_mix, train_rushes_sedges]
test_frames = [test_shrub_sphagnum, test_water, test_sphagnum_r, test_pool_bogbean, test_calluna_mix, test_rushes_sedges]


train = pd.concat(train_frames)
test = pd.concat(test_frames)

# cross_lochs_gdf.PFT.unique()

In [ ]:
# initial dataset

# x_train = train[train.columns[1:359]]
# y_train = train[train.columns[362]]

# x_test = test[test.columns[1:359]]
# y_test = test[test.columns[362]]

# x_test.head(2)


#derivatives dataset

x_train = train[train.columns[1:359]]
y_train = train[train.columns[359]]

x_test = test[test.columns[1:359]]
y_test = test[test.columns[359]]

x_test.head(2)

In [ ]:
from sklearn import dummy

dc = dummy.DummyClassifier()
dc.fit(x_train, y_train)
dc_score = dc.score(x_test, y_test)

print("Baseline Accuracy:", dc_score)


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(max_depth=5, random_state=42)
# fit to the training data
clf.fit(x_train, y_train)
print(clf.score(x_train, y_train))

# print(clf.predict(x_test))


# predict using the model
clf_train_pred = clf.predict(x_train)
clf_pred = clf.predict(x_test)

# append to the results dataframe
# model_results = model_results.assign(rf_pred = rf_pred)
# print(model_results)

# test the performance of the model
scores = cross_val_score(clf, x_train, y_train, cv=3)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


In [ ]:
test_score = cross_val_score(clf, x_test, y_test, cv=3)
print("%0.2f accuracy with a standard deviation of %0.2f" % (test_score.mean(), test_score.std()))

In [ ]:
# initial dataset

# model_results = test[test.columns[360:363]]

# derivatives datasets
model_results = test[test.columns[359:361]]

model_results.head()

In [ ]:
model_results = model_results.assign(clf_pred = clf_pred)

model_results

In [ ]:
# initial dataset

# train_focused = train[train.columns[360:363]]

# derivatives dataset

train_focused = train[train.columns[359:361]]

train_focused.head()

In [ ]:
# model_results.to_file("cross_lochs_results_sample_first_derivative_gdf.shp")

In [ ]:
# train_focused.to_file("cross_lochs_train_sample_first_derivative_gdf.shp")

In [ ]:
# model_results.to_csv("cross_lochs_results_sample_first_derivative_gdf.csv")
# train_focused.to_csv("cross_lochs_train_sample_gdf.csv")

In [ ]:
# now want to predict for the whole dataset using the random forest
# cross_lochs = pd.read_csv('../cross_lochs_gdf.csv')
cross_lochs = pd.read_csv('../transformed_spectral_signatures/first_derivative_full_cross_lochs.csv')

cross_lochs['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs['geometry'])

cross_lochs_gdf = geopandas.GeoDataFrame(cross_lochs, geometry='geometry')

cross_lochs_gdf

In [ ]:
x = cross_lochs_gdf[cross_lochs_gdf.columns[1:359]]

x.head()

In [ ]:
clf_pred_full = clf.predict(x)

clf_pred_full

In [ ]:
# initial dataset

# model_results_full = cross_lochs_gdf[cross_lochs_gdf.columns[360:362]]

# derivatives

model_results_full = cross_lochs_gdf[cross_lochs_gdf.columns[359:361]]

model_results_full = model_results_full.assign(clf_pred_full = clf_pred_full)

model_results_full.head()

In [ ]:
model_results_full.to_file("cross_lochs_results_98_sample_full_first_derivative_0.3_gdf.shp")

model_results_full.to_csv("cross_lochs_results_98_sample_full_first_derivative_0.3_gdf.csv")
